Import Libraries


In [1]:
import numpy as np
print (np.__version__)
from osgeo import gdal
from scipy import stats
import timeit

1.11.3


<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [2]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
g = np.array(r.GetRasterBand(1).ReadAsArray())

In [3]:
## I checked the top left cell in q gis and it is value = 95 bottom left = 41. I want g[0,0] = 41 so that the bottom left is 0,0
print(g[0,0])
g = np.flipud(g)
print(g[0,0])

95
41


In [4]:
##stats.describe(g) 
print( (r.RasterYSize,r.RasterXSize))
print( g.shape )
##out.shape

(1734, 2501)
(1734, 2501)


Function to create data structure function takes in a grid with mxn and returns a grid of the same size summed by columns.  A vectorized version would be faster

In [6]:
def dtFZS(g,out):
    for xw in range(0,g.shape[1]):
        for yw in range(1,g.shape[0]):
            out[yw,xw] = out[yw-1,xw] + g[yw,xw]
    print("done")

In [7]:
##out = dtFZS(g)
DS_FstZS = np.zeros(g.shape,dtype='uint64')
dtFZS(g,DS_FstZS)


done


In [8]:
stats.describe(DS_FstZS)
##out.shape

DescribeResult(nobs=1734, minmax=(array([0, 0, 0, ..., 0, 0, 0], dtype=uint64), array([ 89011,  90144,  90586, ..., 113572, 112982, 113624], dtype=uint64)), mean=array([ 49862.07381776,  50629.15109573,  50998.32583622, ...,
        58392.72029988,  58344.16897347,  58694.10149942]), variance=array([  7.32742147e+08,   7.43157142e+08,   7.51319909e+08, ...,
         1.12376861e+09,   1.11718111e+09,   1.11676096e+09]), skewness=array([-0.20865189, -0.21935293, -0.21832481, ..., -0.09410791,
       -0.09877856, -0.10030696]), kurtosis=array([-1.25720461, -1.238101  , -1.22930801, ..., -1.22156643,
       -1.22243666, -1.20527447]))

Fast Summary Statistics Function
    

In [16]:
## Create Tuple function
def xyrange(ystart,ystop,xstart,xstop,step=1):
    if(xstop < xstart):
        step = - 1
    xtuple = list(range(xstart,xstop,step))
    ytuple = [round(( x/len(xtuple) * (ystop-ystart) ) + ystart) for x in range(0,len(xtuple))]  
    return [ytuple,xtuple]

idx = xyrange(0,100,10,5)
print(idx)
#idx = [(0,0,1,1),(0,1,0,2)]

DS_FstZS[idx]
 

[[0, 20, 40, 60, 80], [10, 9, 8, 7, 6]]


array([   0,  831, 2099, 3437, 4812], dtype=uint64)

In [17]:
##  To DO
## This function needs to be rewritten to make v a lattice on the grid (not matching the grid locations but between the grid valeus)
## and to take coordinates

def FstZS(v,DS_FstZS):
    total = 0;    
    for p in range(1,(len(v)+0)):
        ## If moving left
        if v[p][1] > v[p-1][1]:            
            total += np.sum(DS_FstZS[xyrange(v[p][0],v[p-1][0],v[p][1],v[p-1][1])])
        ## If moving Right
        elif v[p][1] < v[p-1][1]: 
            total -= np.sum(DS_FstZS[xyrange(v[p][0],v[p-1][0],v[p][1],v[p-1][1])])     
    return int(total)

In [18]:
## Test Data set
z = 1000
## Create test box z in size
##v = [(x,1) for x in range(1,z+1)] + [(z,x) for x in range(1+1,z+1)] + [(x,z) for x in range(z-1,0,-1)] + [(1,x) for x in range(z-1,0,-1)]
v = [(0,0),(z,0),(z,z),(0,z),(0,0)]
v
 

[(0, 0), (1000, 0), (1000, 1000), (0, 1000), (0, 0)]

In [20]:
#g.shape
FstZS(v,DS_FstZS)
##xyrange

53453962

In [21]:
##%timeit 'np.sum(g[1:1001,1:1001])'
np.sum(g[1:1001,1:1001])

53453962

Pretty Close Not sure why these are slightly different end